# Insights obtenidos del modelo

Esta notebook contiene un análisis de las *features* aprendidas por el modelo, con el fin de extraer *insights* que puedan ser de utilidad para el negocio

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

## Lectura de datos

In [ ]:
df_data = pd.read_csv('dataset_balanceado.csv')
df_data.head()

## Dataset entrenamiento y test

A continuación se define una función que divide el dataset en dos: uno para entrenamiento del modelo y otro para la evaluación del mismo

In [ ]:
y = df_data['target']
X = df_data.drop(columns=['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

## Definición y entrenamiento del modelo

Se entrena un clasificador *AdaBoost*, que fue aquel que tuvo mejor rendimiento

In [ ]:
clf = AdaBoostClassifier()
clf.fit(X_train, y_train)

## Análisis de los features utilizados por el modelo

Para poder obtener *insights* que puedan ser de utilidad para el negocio se analizan la importancia que el modelo le asigna a cada uno de los *features* 

In [ ]:
feature_importance = clf.feature_importances_
features_importance_serie = pd.Series(data=feature_importance, index=X.columns)

Primero se descartan aquellas *features* cuya prioridad es igual a 0.0; es decir aquellas que el modelo ignora porque no le aportan ningún tipo de valor. 

Es importante tener en cuenta estas *features* a la hora de optimizar el modelo, porque son *features* que es preferible descartar ya que pueden introducir ruido en el modelo.

In [ ]:
features_importance_serie[features_importance_serie == 0]

Luego se analizan la prioridad que el modelo le asignó a cada una de las *features*.

Aquí se puede observar la importancia que tiene la *feature* **total_events_on_Web** que describe la cantidad de eventos que tuvo el *trial* por una computadora. Esta es la variable que más influye a la hora de tomar la decisión por parte del modelo (alrededor del 40%).

Asimismo, se puede observar que hay un grupo de *features* que están en segundo orden de importancia para el modelo. Dentro de este grupo se encuentran las *features*:
* **total_products_with_images**, que describe la cantidad de productos que poseen imagen, con un coeficiente de influencia que ronda 10%;
* **creation_platform**, que describe la plataforma por la cual se creó la tienda, con un coeficiente cercano al 5%. Específicamente, se observa que la categoría *desktop* posee una mayor preponderancia respecto de los otros dispositivos.
* **source_pulido**, que describe el origen de la tienda por campaña de marketing, con un coeficiente cercano también al 5%. Específicamente, se observa que la categoría *Facebook CPC* posee una mayor preponderancia respecto de las otras tiendas campañas.

In [ ]:
features_importance_serie = features_importance_serie[features_importance_serie > 0]

plt.figure(figsize=(15, 12))

ax = sns.barplot(x=features_importance_serie.index, y=features_importance_serie)

ax.set_ylabel('feature importance', fontsize=25)
ax.set_xticklabels(features_importance_serie.index, rotation=80, fontsize=14)